### 节点：
歌手（姓名、性别、国籍）  
歌曲（歌曲名、发行时间、时长、歌词、点击量）  

专辑（专辑名、发行时间、简介）  

歌单（歌单名、简介、建立时间）   

流派（流派名）   
用户（用户名、密码、注册时间）   

### 关系类型：
歌手——歌曲：演唱   
歌手——专辑：发行   
专辑——歌曲：包括    
用户——歌单：创建    
歌曲——流派：属于    
歌单——歌曲：包含   


In [1]:
from py2neo import Graph
graph = Graph("bolt: // localhost:7687", auth=("neo4j", "12345678"))

from py2neo import Graph, Node, Relationship
from py2neo.matching import NodeMatcher

#查询节点函数
def get_nodes(label, **kwargs):
    matcher = NodeMatcher(graph)
    return matcher.match(label, **kwargs)

#### 热歌榜（根据点击量）


In [2]:
#查询函数，显示歌曲的歌曲名，发行时间，时长，点击量，根据歌曲的点击量排序
def query_songs_by_clicks():
    query = """
    MATCH (song:歌曲)
    RETURN song.名字 AS 歌曲名, song.发行时间 AS 发行时间, song.时长 AS 时长, song.点击量 AS 点击量
    ORDER BY song.点击量 DESC
    LIMIT 30
    """
    result = graph.run(query)
    for record in result:
        song_name = record["歌曲名"]
        release_time = record["发行时间"]
        duration = record["时长"]
        clicks = record["点击量"]
        print(f"歌曲名: {song_name}, 发行时间: {release_time}, 时长: {duration}, 点击量: {clicks}")

#更新函数，点击喜欢之后点击量更新为增加一
def update_song_clicks(song_name):
    query = f"""
    MATCH (song:歌曲 {{名字: '{song_name}'}})
    SET song.点击量 = song.点击量 + 1
    RETURN song.点击量 AS 点击量
    """
    result = graph.run(query).evaluate()
    if result:
        print(f"歌曲'{song_name}'的点击量已更新为 {result}")
    else:
        print(f"未找到歌曲'{song_name}'")


In [3]:
#实例化测试
resou= query_songs_by_clicks()
resou

歌曲名: 七里香, 发行时间: 2000-10-10, 时长: 3:20, 点击量: 301
歌曲名: 听海, 发行时间: 2002-10-10, 时长: 3:50, 点击量: 200
歌曲名: lover, 发行时间: 2013-10-10, 时长: 4:20, 点击量: 100


In [4]:
#测试点击量更新函数
click=update_song_clicks('七里香')

歌曲'七里香'的点击量已更新为 302


#### 流派分类（风格类型，点击跳转该流派所有歌曲）


In [5]:
#查询函数，查询与该流派节点有关系的所有歌曲
def query_songs_by_genre(genre):
    query = f"""
    MATCH (genre:流派 {{流派名: '{genre}'}})<-[:属于]-(song:歌曲)
    RETURN song.名字 AS 歌曲名, song.发行时间 AS 发行时间, song.时长 AS 时长, song.点击量 AS 点击量
    """
    result = graph.run(query)
    for record in result:
        song_name = record["歌曲名"]
        release_time = record["发行时间"]
        duration = record["时长"]
        clicks = record["点击量"]
        print(f"歌曲名: {song_name}, 发行时间: {release_time}, 时长: {duration}, 点击量: {clicks}")


In [6]:
#测试流派分类
liupai = query_songs_by_genre('流行')

歌曲名: 七里香, 发行时间: 2000-10-10, 时长: 3:20, 点击量: 302
歌曲名: 听海, 发行时间: 2002-10-10, 时长: 3:50, 点击量: 200
歌曲名: lover, 发行时间: 2013-10-10, 时长: 4:20, 点击量: 100


#### 创建用户

In [35]:
#增加函数，增加用户节点
def create_user_node(user_name, create_date=None, password=None,):
        properties = {"名字": user_name}
        if create_date is not None:
            properties["注册日期"] = create_date
        if password is not None:
            properties["密码"] = password

        node = Node('用户', **properties)
        graph.create(node)
        return node

In [36]:
cuser=create_user_node('user1','2023-01-01','1234')

#### “我的”：我喜欢（加入我喜欢和移除），创建歌单（增加歌单和删除歌单的功能）


In [105]:
#增加函数，增加歌单与对应歌曲的关系，条件是歌单与用户有创建关系
def create_song_list_relationship(user_name, song_name, rel_type, list_name, properties):
    start_node = None
    end_node = None
    user_node = None

    song_nodes = get_nodes("歌曲", 名字=song_name)
    for song_node in song_nodes:
        end_node = song_node

    list_nodes = get_nodes("歌单", 歌单名=list_name)
    for list_node in list_nodes:
        start_node = list_node
    
    user_nodes = get_nodes('用户', 名字=user_name)
    for user_node in user_nodes:
        user_node = user_node

    if start_node and end_node:
        user_list_rel = get_user_list_relationship(user_name, list_name)
        if user_list_rel is not None:
            relationship = Relationship(start_node, rel_type, end_node, **properties)
            graph.create(relationship)
            print("关系创建成功")
            return relationship
        else:
            print("用户与歌单没有关系")
            return None
    else:
        print("未找到对应的节点")
        return None

def get_user_list_relationship(user_name, list_name):
    query = f"""
    MATCH (:用户 {{名字: '{user_name}'}})-[r]->(:歌单 {{歌单名: '{list_name}'}})
    RETURN r
    """
    result = graph.run(query).evaluate()
    return result


In [111]:
#测试增加歌曲到歌单函数
zenggedan=create_song_list_relationship('user1','lover','包含','新的歌单1',{'关系类型':'包含'})

关系创建成功


In [82]:

#删除函数，删除歌单与对应歌曲的关系，条件是歌单与用户有创建关系
def delete_song_list_relationship(user_name, song_name, list_name):
    song_node = None
    list_node = None
    user_node = None

    song_nodes = get_nodes("歌曲", 名字=song_name)
    for node in song_nodes:
        song_node = node

    list_nodes = get_nodes("歌单", 歌单名=list_name)
    for node in list_nodes:
        list_node = node

    user_nodes = get_nodes('用户', 名字=user_name)
    for node in user_nodes:
        user_node = node

    if song_node and list_node:
        user_list_rel = get_user_list_relationship(user_name, list_name)
        if user_list_rel is not None:
            query = """
            MATCH (s1:歌单)-[r:包含]->(s2:歌曲)
            WHERE s1.歌单名 = $list_name AND s2.名字 = $song_name
            DELETE r
            """
            result = graph.run(query, list_name=list_name, song_name=song_name)
            result.evaluate()
            print("关系删除成功")
        else:
            print("用户与歌单没有关系")
    else:
        print("未找到对应的节点")


In [109]:
shan1= delete_song_list_relationship('user1','听海','新的歌单1')

关系删除成功


In [11]:

#增加函数，增加歌单节点，并且增加客户与歌单节点的关系
def create_list_node(list_name, create_date=None, brief_info=None,):
        properties = {"歌单名": list_name}
        if create_date is not None:
            properties["创建日期"] = create_date
        if brief_info is not None:
            properties["简介"] = brief_info

        node = Node('歌单', **properties)
        graph.create(node)
        return node

def create_user_list_relationship(user_name, rel_type, list_name, properties):
    start_node = None
    end_node = None

    nodes = get_nodes("用户", 名字=user_name)
    for node in nodes:
        start_node = node
        print(start_node)

    nodes = get_nodes("歌单", 歌单名=list_name)
    for node in nodes:
        end_node = node
        print(end_node)

    if start_node and end_node:
        relationship = Relationship(start_node, rel_type, end_node, **properties)
        graph.create(relationship)
        print("关系创建成功")
        return relationship
    else:
        print("未找到对应的节点")
        return None
    

In [32]:
#测试增加歌单函数
add_list =create_list_node('歌单2','2023-1-2','歌单2的简介')
add_list

Node('歌单', 创建日期='2023-1-2', 歌单名='歌单1', 简介='歌单2的简介')

In [38]:
#测试增加用户和歌单的关系
user_list = create_user_list_relationship('user1','创建','歌单1',{'关系类型':'创建'})

(_20:用户 {名字: 'user1', 密码: '1234', 注册日期: '2023-01-01'})
(_19:歌单 {创建日期: '2023-1-2', 歌单名: '\u6b4c\u53551', 简介: '\u6b4c\u53552\u7684\u7b80\u4ecb'})
关系创建成功


In [93]:
#删除歌单节点，并且删除该歌单的关系，包括用户节点与歌单的关系，歌曲节点与歌单的关系
def delete_list_node(playlist_name):
    playlist_node = None

    nodes = get_nodes("歌单", name=playlist_name)
    for node in nodes:
        playlist_node = node

    if playlist_node:
        delete_relationships_with_user(playlist_name)
        delete_relationships_with_other_nodes(playlist_name)
        graph.delete(playlist_node)
        print("歌单节点删除成功")
    else:
        print("未找到对应的节点")

def delete_relationships_with_other_nodes(playlist_name):
    query = f"MATCH (p:歌单)-[r]-() WHERE p.歌单名 = '{playlist_name}' DELETE r"
    graph.run(query)

def delete_relationships_with_user(playlist_name):
    query = f"MATCH (u:用户)-[r]->(p:歌单) WHERE p.歌单名 = '{playlist_name}' DELETE r"
    graph.run(query)


In [95]:
dpl=delete_list_node('歌单2')

歌单节点删除成功


#### 查找：跳转一个搜索页面，歌手（歌手、三张专辑、十首歌）、歌名（歌手、专辑、类似歌曲）、专辑名


In [46]:
#查询函数，根据查询信息给出对应节点信息
#查询歌手时，显示该歌手节点的信息，显示与该歌手有关系的三个专辑节点信息，显示与该歌手节点有关系的十个歌曲节点信息，按点击量排序
def search_artist( artist_name):
    query = f"""
    MATCH (artist:歌手 {{名字: '{artist_name}'}})
    OPTIONAL MATCH (artist)-[:发行]->(album:专辑)
    OPTIONAL MATCH (artist)-[:演唱]->(song:歌曲)
    RETURN artist.名字 AS 姓名, artist.性别 AS 性别, artist.国籍 AS 国籍,
           album.名字 AS 专辑名, album.发行时间 AS 专辑发行时间, album.简介 AS 专辑简介,
           song.名字 AS 歌曲名, song.发行时间 AS 歌曲发行时间, song.时长 AS 歌曲时长, song.点击量 AS 点击量
    ORDER BY song.点击量 DESC
    LIMIT 10
    """
    result = graph.run(query)
    for record in result:
        artist_name = record["姓名"]
        gender = record["性别"]
        nationality = record["国籍"]
        album_name = record["专辑名"]
        album_release_time = record["专辑发行时间"]
        album_description = record["专辑简介"]
        song_name = record["歌曲名"]
        song_release_time = record["歌曲发行时间"]
        song_duration = record["歌曲时长"]
        clicks = record["点击量"]
        
        print(f"歌手姓名: {artist_name}, 性别: {gender}, 国籍: {nationality}")
        print(f"相关专辑:")
        print(f"专辑名: {album_name}, 发行时间: {album_release_time}, 简介: {album_description}")
        print(f"相关歌曲:")
        print(f"歌曲名: {song_name}, 发行时间: {song_release_time}, 时长: {song_duration}, 点击量: {clicks}")
        print("----")


In [100]:
#测试歌手信息查询
sat= search_artist('周杰伦')

歌手姓名: 周杰伦, 性别: 男, 国籍: 中国
相关专辑:
专辑名: 七里香, 发行时间: 2000-10-10, 简介: 七里香专辑的简介
相关歌曲:
歌曲名: 七里香, 发行时间: 2000-10-10, 时长: 3:20, 点击量: 302
----


In [48]:

#查询歌名时，显示该歌曲的信息，并且显示与该歌曲有关系的歌手名称，显示与该歌曲有关系的专辑名称，显示与该歌曲有关系的歌单名称
def search_song(song_name):
    query = f"""
    MATCH (song:歌曲 {{名字: '{song_name}'}})
    OPTIONAL MATCH (song)<-[:演唱]-(artist:歌手)
    OPTIONAL MATCH (song)<-[:发行]-(album:专辑)
    OPTIONAL MATCH (song)<-[:包含]-(playlist:歌单)
    RETURN song.名字 AS 歌曲名, song.发行时间 AS 发行时间, song.时长 AS 时长, song.歌词 AS 歌词,
           artist.名字 AS 歌手, album.专辑名 AS 专辑, playlist.歌单名 AS 歌单
    """
    result = graph.run(query)
    for record in result:
        song_name = record["歌曲名"]
        release_time = record["发行时间"]
        duration = record["时长"]
        lyrics = record["歌词"]
        artist = record.get("歌手")
        album = record.get("专辑")
        playlist = record.get("歌单")
        
        print(f"歌曲名: {song_name}")
        print(f"发行时间: {release_time}")
        print(f"时长: {duration}")
        print(f"歌词: {lyrics}")
        
        if artist:
            print(f"歌手: {artist}")
        
        if album:
            print(f"专辑: {album}")
        
        if playlist:
            print(f"歌单: {playlist}")
        
        print("----")


In [96]:
ssong=search_song('七里香')

歌曲名: 七里香
发行时间: 2000-10-10
时长: 3:20
歌词: 七里香的歌词
歌手: 周杰伦
歌单: 新的歌单1
----


In [50]:

#查询专辑名时，显示该专辑的信息，并且显示该专辑有关系的歌手名称，显示与该专辑节点有关系的所有歌曲节点
def search_album(album_name):
    query = f"""
    MATCH (album:专辑 {{名字: '{album_name}'}})
    OPTIONAL MATCH (album)<-[:发行]-(artist:歌手)
    OPTIONAL MATCH (album)<-[:包含]-(song:歌曲)
    RETURN album.名字 AS 专辑名, album.发行时间 AS 发行时间, album.简介 AS 简介,
           artist.名字 AS 歌手, song.歌曲名 AS 歌曲名, song.发行时间 AS 歌曲发行时间, song.时长 AS 歌曲时长
    """
    result = graph.run(query)
    for record in result:
        album_name = record["专辑名"]
        release_time = record["发行时间"]
        description = record["简介"]
        artist = record.get("歌手")
        song_name = record.get("歌曲名")
        song_release_time = record.get("歌曲发行时间")
        song_duration = record.get("歌曲时长")
        
        print(f"专辑名: {album_name}")
        print(f"发行时间: {release_time}")
        print(f"简介: {description}")
        
        if artist:
            print(f"歌手: {artist}")
        
        if song_name:
            print("相关歌曲:")
            print(f"歌曲名: {song_name}, 发行时间: {song_release_time}, 时长: {song_duration}")
        
        print("----")


In [97]:
sablum=search_album('七里香')

专辑名: 七里香
发行时间: 2000-10-10
简介: 七里香专辑的简介
----


#### 用户可以修改歌单名。（用户可以关注歌手）

In [52]:
#更新函数，用户对应歌单名更新为输入参数
def update_list_name( old_playlist_name, new_playlist_name):
    query = f"""
    MATCH (playlist:歌单 {{歌单名: '{old_playlist_name}'}})
    SET playlist.歌单名 = '{new_playlist_name}'
    RETURN playlist
    """
    result = graph.run(query)
    if result:
        print(f"歌单名称已成功更新为 '{new_playlist_name}'")
    else:
        print("未找到指定的歌单节点")


In [55]:
#测试更新歌单名
upln= update_list_name('歌单1','新的歌单1')

歌单名称已成功更新为 '新的歌单1'


In [106]:

#增加函数，增加用户与歌手直接的关注关系
def create_user_artist_relationship( user_name, artist_name):
    user_node = None
    artist_node = None

    user_nodes = get_nodes("用户", 名字=user_name)
    for node in user_nodes:
        user_node = node

    artist_nodes = get_nodes("歌手", 名字=artist_name)
    for node in artist_nodes:
        artist_node = node

    if user_node and artist_node :
        relationship = Relationship(user_node, "关注", artist_node)
        graph.create(relationship)
        print(f"用户'{user_name}'已关注歌手'{artist_name}'")
    else:
        print("未找到对应的用户或歌手节点")

In [108]:
#测试用户关注歌手
car=create_user_artist_relationship('user1','张学友')

(_20:用户 {名字: 'user1', 密码: '1234', 注册日期: '2023-01-01'})
(_6:歌手 {名字: '\u5f20\u5b66\u53cb', 国籍: '\u4e2d\u56fd', 性别: '\u7537'})
用户'user1'已关注歌手'张学友'


#### 详情页显示该歌曲的全部信息，并且推荐按钮显示该歌曲的相关歌曲

In [64]:
#查询函数，显示某歌曲的全部信息
def query_song_info( song_name):
    query = f"""
    MATCH (song:歌曲 {{名字: '{song_name}'}})
    RETURN song
    """
    result = graph.run(query)
    for record in result:
        song_info = record["song"]
        print(song_info)


In [65]:
qsi = query_song_info('七里香')

(_9:歌曲 {发行时间: '2000-10-10', 名字: '\u4e03\u91cc\u9999', 时长: '3:20', 歌词: '\u4e03\u91cc\u9999\u7684\u6b4c\u8bcd', 点击量: 302})


In [113]:

#查询函数，显示和某歌曲在同一歌单节点，在同一专辑节点，在同一流派节点的歌曲，按照热度排序，显示前10首歌曲
def recommend_songs(song_name):
    query = f"""
    MATCH (song:歌曲 {{名字: '{song_name}'}})
    OPTIONAL MATCH (song)<-[:包含]-(:歌单)-[:包含]->(recommended:歌曲)
    OPTIONAL MATCH (song)<-[:属于]-(:流派)-[:属于]->(recommended:歌曲)
    OPTIONAL MATCH (song)<-[:包括]-(:专辑)-[:包括]->(recommended:歌曲)
    WHERE recommended <> song
    RETURN recommended.名字 AS 歌曲名, recommended.发行时间 AS 发行时间, recommended.时长 AS 时长, recommended.点击量 AS 点击量
    ORDER BY recommended.点击量 DESC
    LIMIT 10;
    """
    result = graph.run(query)
    for record in result:
        song_name = record["歌曲名"]
        release_time = record["发行时间"]
        duration = record["时长"]
        clicks = record["点击量"]
        print(f"歌曲名: {song_name}, 发行时间: {release_time}, 时长: {duration}, 点击量: {clicks}")


In [114]:
recommend = recommend_songs('七里香')

歌曲名: 听海, 发行时间: 2002-10-10, 时长: 3:50, 点击量: 200
歌曲名: lover, 发行时间: 2013-10-10, 时长: 4:20, 点击量: 100
